In [1]:
from aam.data_handlers.generator_dataset import GeneratorDataset

2025-01-24 17:01:40.845234: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-24 17:01:40.845359: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-24 17:01:40.849615: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-24 17:01:41.617955: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/swchan/.conda/envs/aam/lib/python3.9/site-pa

In [2]:
import tensorflow as tf
class ASVWrapper(tf.keras.layers.Layer):
    def __init__(self, asv_encoder, **kwargs):
        super(ASVWrapper, self).__init__(**kwargs)
        self.asv_encoder = asv_encoder
        self.asv_encoder.trainable = False
        self.embedding_dim = self.asv_encoder.embedding_dim
    def _batch_embeddings(self, asv_embeddings, batch_indicies, counts):
        batch_shape = tf.reduce_max(batch_indicies[:, 0]) + 1
        max_unique = tf.reduce_max(batch_indicies[:, 1]) + 1
        batch_embeddings = tf.scatter_nd(batch_indicies, asv_embeddings, shape=[batch_shape, max_unique, self.embedding_dim])
        counts = tf.scatter_nd(batch_indicies, counts, shape=[batch_shape, max_unique, 1])
        return batch_embeddings, counts
    def call(self, inputs, training=False):
        tokens, batch_indicies, asv_indicies, counts = inputs
        batch_indicies = tf.cast(batch_indicies, dtype=tf.int32)
        asv_indicies = tf.cast(asv_indicies, dtype=tf.int32)
        asv_embeddings = self.asv_encoder(tokens, training=training)
        asv_embeddings = tf.gather(asv_embeddings, asv_indicies)
        return self._batch_embeddings(asv_embeddings, batch_indicies, counts)
    def get_config(self):
        config = super().get_config()
        config.update({"asv_encoder": tf.keras.saving.serialize_keras_object(self.asv_encoder)})
    @classmethod
    def from_config(cls, config):
        config["asv_encoder"] = tf.keras.saving.deserialize_keras_object(config["asv_encoder"])
        model = cls(**config)
        return model

In [3]:
def get_dataset(gen):
    enqueuer = tf.keras.utils.OrderedEnqueuer(gen, use_multiprocessing=True)
    enqueuer.start(workers=2, max_queue_size=gen.steps_per_epoch)
    gen.stop = lambda: enqueuer.stop(0.1)
    y_type = tf.TensorSpec(shape=(gen.samples_per_minibatch, 1), dtype=tf.string)
    dataset = tf.data.Dataset.from_generator(
        enqueuer.get,
        output_signature=(
            (
                tf.TensorSpec(shape=[None, 150], dtype=tf.int32),
                tf.TensorSpec(shape=[None, 2], dtype=tf.int32),
                tf.TensorSpec(shape=[None], dtype=tf.int32),
                tf.TensorSpec(shape=[None, 1], dtype=tf.int32),
            ),
            tf.TensorSpec(shape=(gen.batch_size, 1), dtype=tf.string),
        ),
    )
    dataset = dataset.prefetch(10)
    return dataset

In [4]:
gd = GeneratorDataset(
    table='/home/swchan/DSC170/fecal.biom',
    metadata='/home/swchan/DSC170/fecal.tsv',
    metadata_column='covid_positive',
    shuffle=False,
    is_categorical=True
)

Unrarefied table shape: (7969, 60)
metadata shape: (60,)


In [5]:
dataset = get_dataset(gd)

2025-01-24 17:02:06.637213: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10534 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:86:00.0, compute capability: 6.1


In [6]:
# import pandas as pd
# from biom import load_table
# df = ""

# df["covid_positive"]

# table = load_table("fecal.biom")
# df = df.sort_index(table.ids())

In [7]:
# import tensorflow as tf
# model = tf.keras.models.load_model('/home/swchan/DSC170/model.keras', compile=False)
# model = ASVWrapper(model)

# @tf.function(reduce_retracing=True)
# def call_step(inputs):
#     return model(inputs)
                 
# for x, y in dataset.take(gd.steps_per_epoch):
#     embeddings, counts = call_step(x)
#     attention_mask = tf.cast(counts  >  0, dtype=tf.float32)
#     attention_mask = tf.matmul(attention_mask, attention_mask, transpose_b=True)
#     print("k")
# gd.stop()

#Output B x A x E Tensor (Sample Number x ASVs x Embedding Dimension)
# B x A x E Tensor (Sample Number x ASVs x 1)

In [9]:
import tensorflow as tf
base_model = tf.keras.models.load_model('/home/swchan/DSC170/model.keras', compile=False)
base_model = ASVWrapper(base_model)

create asv layer with 8 heads
Encoder residual connection...
Encoder exit... True
ASVEncoder exit... True
Pooler residual connection...
Pooler exit... True
BaseSequenceEncoder exit... True


In [11]:
K = tf.keras

In [12]:
class TransformerEncodingBlock(K.layers.Layer):
    def __init__(self, num_attention_heads=4, dropout_rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.num_attention_heads = num_attention_heads
        self.dropout_rate = dropout_rate

    def build(self, input_shape):
        emb_dim = input_shape[-1]
        key_dim = emb_dim // self.num_attention_heads
        self.attention_layer = K.layers.MultiHeadAttention(self.num_attention_heads, key_dim=key_dim, dropout=self.dropout_rate)
        self.ff = K.layers.Dense(emb_dim)
        self.rezero_alpha = self.add_weight("rezero_alpha", 1, dtype="float32", initializer="zero", trainable=True)

    def call(self, input, attention_mask=None, training=False):
        """
        input: [B, A, N] float
        attention_mask: [B, A, 1] bool
        """
        # attention
        attention_mask = tf.matmul(attention_mask, attention_mask, transpose_b=True)  # [B, A, A]
        attention_output = self.attention_layer(input, input, attention_mask=attention_mask, training=training)

        # rezero skip connection
        attention_output = input + self.rezero_alpha * attention_output

        ff_output = self.ff(attention_output)

        return attention_output + self.rezero_alpha * ff_output

In [13]:
class TransformerEncoder(K.layers.Layer):
    def __init__(self, num_attention_layers=4, num_attention_heads=4, dropout_rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.num_attention_layers = num_attention_layers
        self.num_attention_heads = num_attention_heads
        self.dropout_rate = dropout_rate

        self.encoding_layers = [
            TransformerEncodingBlock(self.num_attention_heads, self.dropout_rate) for _ in range(self.num_attention_layers)
        ]
        self.rezero_alpha = self.add_weight("rezero_alpha", 1, dtype="float32", initializer="zero", trainable=True)

    def call(self, inputs, attention_mask=None, training=False):
        output = inputs
        for i in range(self.num_attention_layers):
            output = self.encoding_layers[i](output, attention_mask=attention_mask, training=training)
        return inputs * self.rezero_alpha * output

In [14]:
K=tf.keras
class Classifier(K.Model):
    def __init__(self, feature_extractor, **kwargs):
        super().__init__(**kwargs)

        self.feature_extractor = feature_extractor
        self.feature_extractor.trainable = False

        self.encoder = TransformerEncoder()  # transformer (or anything else)
        self.pooling = lambda input: K.backend.mean(input, axis=-1)  # extracts sample embeddings from our ASV embeddings
        self.tokenizer = K.layers.TextVectorization(4, split="character", pad_to_max_tokens=True)
        self.dense_ff = K.layers.Dense(2, activation="softmax")  # sample_embedings => classifys covid +/-

        self.loss = K.losses.BinaryCrossentropy()

        self.auc_metric = K.metrics.AUC()
        self.loss_tracker = K.metrics.Mean()

    def build(self, input_shape):
        pass

    def call(self, inputs, mask=None, training=False):
        """
        inputs: [B, A, N], B: batch_dim, A: # ASV in sample, N: nuctides,
        string tensor
        """
        features = self.feature_extractor(inputs)  # [B, A, N, E], will probably be different for all models

        # some addional processing
        if isinstance(self.feature_extractor, "DNABert"):
            # do something
            features = self._extract_asv_embeddings_from_bert(features)
            pass
        elif isinstance(self.feature_extractor, "Grove"):
            # do something else
            features = self._extract_asv_embeddings_from_grover(features)
            pass
        else:
            # aam case
            features = self._extract_asv_embeddings_from_aam(features)
            pass

        encodig_output = self.encoder(features, attention_mask=mask, training=training)  # [B, A, N]
        pooling_output = self.pooling(encodig_output)  # [B, N]
        return self.dense_ff(pooling_output)

    def train_step(self, data):
        x, y = data

        # create attention mask
        # example [["ACTG"], [""]]
        mask = K.backend.cast(K.backend.sum(self.tokenizer(x), axis=-1) == 0, dtype="float32")
        with tf.GradientTape() as tape:
            output = self(x, mask=mask, training=True)
            loss = self.loss(y, output)

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        self.loss_tracker.update_state(loss)
        self.auc_tracker.update_state(y, output)
        return {"loss": self.loss_tracker.result(), "auc": self.auc_tracker.result()}

    def test_step(self, data):
        x, y = data

        mask = K.backend.cast(K.backend.sum(self.tokenizer(x), axis=-1) == 0, dtype="float32")
        output = self(x, mask=mask, training=False)
        loss = self.loss(y, output)

        self.loss_tracker.update_state(loss)
        self.auc_tracker.update_state(y, output)

        return {"loss": self.loss_tracker.result(), "auc": self.auc_tracker.result()}

    def predict_step(self, data):
        x, y = data
        return self(x)

In [15]:
model = Classifier(base_model)

optimizer = K.optimizers.Adam(
    learning_rate=1e-4
)

In [17]:
x, y = gd[0]

In [18]:
model.build([None, None, 1])
model.compile(loss=K.losses.BinaryCrossentropy())